In [ ]:
%load_ext autoreload
%autoreload 2
# Enable imports form top-level of project (edit top_level_path accordingly)
import os
import sys
top_level_path = os.path.abspath(os.path.join('..'))
if top_level_path not in sys.path:
	sys.path.append(top_level_path)

In [81]:
from pyqubo import Binary, Array, Num
import numpy as np
from dimod import ExactSolver
import neal
from longestpath import gen_average_degree_directed, gen_planted_path, StandardGraph
from typing import List, Tuple, Callable
from math import log2, floor

In [82]:
def make_tail_graph(graph : StandardGraph, tail_length : int) -> StandardGraph:

	if tail_length == 0:
		tail_graph = StandardGraph(0, [])
		tail_graph.vertices += graph.vertices
		tail_graph.edges = graph.edges.copy()
		return tail_graph

	tail_graph = StandardGraph(0, [])
	tail_graph.vertices += graph.vertices + tail_length
	tail_graph.edges = graph.edges.copy()
	
	for i in range(0,graph.vertices):
		tail_graph.edges.append((i,graph.vertices))
	
	for i in range(graph.vertices , graph.vertices + tail_length - 1):
		tail_graph.edges.append((i, i+1))

	return tail_graph


def make_vertex_edges(graph : StandardGraph) -> List[List[Tuple[int, int, int]]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(v_1, v_2) = graph.edges[i]
		
		out[v_1].append((i,v_1,v_2))
		out[v_2].append((i,v_1,v_2))
	
	return out

In [83]:
graph = StandardGraph(5, [
	(0,1),
    (1,2),
    (2,3),
    (3,4),
])

# graph = gen_planted_path(10, 0.05)

# max length
N = graph.vertices
K = N - 1

P = -N**2

#binaries = range(floor(log2(K)) + 1)

tail_graph = make_tail_graph(graph, K)

vertex_edges = make_vertex_edges(graph)

In [84]:


vertex_vars = Array.create('vertex', tail_graph.vertices, vartype='BINARY')
edge_vars = Array.create('edge', len(tail_graph.edges), vartype='BINARY')
initial_edge_vars = Array.create('init_edge', tail_graph.vertices, vartype='BINARY')
terminal_edge_vars = Array.create('term_edge', tail_graph.vertices, vartype='BINARY') 

# flow_vars = Array.create('flow', shape=(len(binaries), len(tail_graph.edges)), vartype='BINARY')
# flow_vars = Array.create('contra_flow', shape=(len(binaries), len(tail_graph.edges)), vartype='BINARY')

#(1)
candy_exp = sum(edge_vars[i] for i in range(len(graph.edges)))

#(2)
initial_node_exp = P * (1 - sum(initial_edge_vars[i] for i in range(graph.vertices)))**2

#(3)
terminal_node_exp = P * (1 - sum(terminal_edge_vars[i] for i in range(tail_graph.vertices)))**2

#(4)
fake_terminal_node_exp_1 = P * (vertex_vars[graph.vertices] - sum(edge_vars[i] for i in range( len(graph.edges), len(graph.edges) + graph.vertices)))**2

fake_terminal_node_exp_2 = P * (1 - vertex_vars[graph.vertices]) * vertex_vars[graph.vertices + 1]

#(5)
real_node_deg_2_exp = 0
for i in range(graph.vertices):
	temp = sum(edge_vars[vertex_edges[i][j][0]] for j in range(len(vertex_edges[i])))
	temp += initial_edge_vars[i] + terminal_edge_vars[i] + edge_vars[graph.vertices + i]
	print(temp)
	real_node_deg_2_exp += P * (2 * vertex_vars[i] - temp)**2

#(6)
tail_node_deg_2_exp = 0
k = 0
for i in range(graph.vertices + 1, tail_graph.vertices):
	tail_node_deg_2_exp += (vertex_vars[i] - edge_vars[len(graph.edges) + k] - edge_vars[len(graph.edges) + k + 1] - terminal_edge_vars[i])**2
tail_node_deg_2_exp = P * tail_node_deg_2_exp





((Binary('edge[5]') + Binary('init_edge[0]') + Binary('term_edge[0]')) + 0.000000 + Binary('edge[0]'))
((Binary('edge[6]') + Binary('init_edge[1]') + Binary('term_edge[1]')) + Binary('edge[1]') + 0.000000 + Binary('edge[0]'))
((Binary('edge[7]') + Binary('init_edge[2]') + Binary('term_edge[2]')) + Binary('edge[2]') + 0.000000 + Binary('edge[1]'))
((Binary('edge[8]') + Binary('init_edge[3]') + Binary('term_edge[3]')) + Binary('edge[3]') + 0.000000 + Binary('edge[2]'))
((Binary('edge[9]') + Binary('init_edge[4]') + Binary('term_edge[4]')) + 0.000000 + Binary('edge[3]'))


In [85]:
total_exp = candy_exp + initial_node_exp + terminal_node_exp + all_node_deg_2_exp

exp = -total_exp

model = exp.compile()
bqm = model.to_bqm()

sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=100)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.energy)

print(best_sample.array)
print(tail_graph.edges)

-4.0
<bound method PyCapsule.array of DecodedSolution({edge[8]:0, term_edge[7]:0, edge[0]:1, term_edge[6]:0, edge[1]:1, term_edge[5]:0, edge[2]:1, term_edge[4]:0, edge[3]:1, term_edge[2]:0, edge[5]:1, term_edge[1]:0, edge[6]:0, term_edge[0]:0, edge[7]:0, edge[9]:0, init_edge[0]:0, init_edge[1]:0, init_edge[2]:0, init_edge[3]:0, init_edge[4]:1, term_edge[3]:0, term_edge[8]:1, vertex[0]:1, vertex[1]:1, vertex[2]:1, vertex[3]:1, vertex[4]:1}, energy=-4.000000)>
[(0, 1), (1, 2), (2, 3), (3, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 6), (6, 7), (7, 8)]
